<a href="https://colab.research.google.com/github/ueki5/colaboratory/blob/main/playground-017.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install plotly | tail -n 1
!pip install japanize-matplotlib | tail -n 1

In [ ]:
import matplotlib.pyplot as plt
import japanize_matplotlib
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np
import pandas as pd


###################################################################
# データ準備
###################################################################
from sklearn.datasets import load_iris

# データ読み込み
iris = load_iris()

# 入力データと正解データ取得
x_org, y_org = iris.data, iris.target

# 結果確認
print('元データ', x_org.shape, y_org.shape)

# データ絞り込み
#  クラス0,1
#  項目 sepal_lengthとsepal_widthのみ
x_data = iris.data[:100, :2]
y_data = iris.target[:100]

# 訓練データ、検証データに分割（シャッフルも同時に実施）
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, train_size=70, test_size=30,
    random_state=123
)

###################################################################
# グラフ描画（配列使用）
###################################################################
x_t0 = x_train[y_train == 0]
x_t1 = x_train[y_train == 1]
# 散布図（setosa）
data1 = go.Scatter(x=x_t0[:,0], y=x_t0[:,1],
                    mode='markers',
                    name='0(setosa)')
# 散布図（versicolor）
data2 = go.Scatter(x=x_t1[:,0], y=x_t1[:,1],
                    mode='markers',
                    name='1(versicolor)')
fig = go.Figure()
fig.add_trace(data1)
fig.add_trace(data2)

fig.update_xaxes(title="sepal_length",
                 color='grey',
                 ticks='inside',
                 ticklen=5,
                 tickwidth=2,
                 tickcolor='lightgrey'
                 ) # X軸タイトルを指定
fig.update_yaxes(title="sepal_width") # Y軸タイトルを指定
fig.update_layout(title="散布図（訓練データ）") # グラフタイトルを設定
fig.update_layout(showlegend=True) # 凡例を強制的に表示（デフォルトでは複数系列あると表示）
fig.show()


元データ (150, 4) (150,)
